<a href="https://colab.research.google.com/github/VNagaHarshitha/SentimentAnalysisProject/blob/main/sentimentanalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import requests
import pandas as pd
from textblob import TextBlob
import getpass, os

# Prompt for API key (hidden input, not stored in notebook)
os.environ["NEWSDATA_API_KEY"] = getpass.getpass("Enter your API key: ")
API_KEY = os.getenv("NEWSDATA_API_KEY")
os.environ["NEWSDATA_URL_KEY"] = getpass.getpass("Enter your URL key: ")
URL_KEY = os.getenv("NEWSDATA_URL_KEY")


# Function to fetch news from API with pagination
def fetch_news(query="business", language="en", max_articles=100):
    all_articles = []
    next_page = None

    while len(all_articles) < max_articles:
        params = {
            "apikey": API_KEY,
            "q": query,
            "language": language,
        }
        if next_page:
            params["page"] = next_page

        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            data = response.json()
            if data.get("status") == "success":
                articles = data.get("results", [])
                all_articles.extend(articles)

                # Check if there's a next page
                next_page = data.get("nextPage")
                if not next_page:
                    break  # No more pages available
            else:
                print("API error:", data)
                break
        else:
            print("Error:", response.status_code, response.text)
            break

    return all_articles[:max_articles]  # Limit to requested number


# Sentiment Analysis Function
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return "Positive"
    elif polarity < 0:
        return "Negative"
    else:
        return "Neutral"


# Predict Market/Business Trend
def predict_market(sentiments):
    pos = sentiments.count("Positive")
    neg = sentiments.count("Negative")
    neu = sentiments.count("Neutral")

    if pos > neg:
        return "Market Outlook: Bullish (Positive trend)"
    elif neg > pos:
        return "Market Outlook: Bearish (Negative trend)"
    else:
        return "Market Outlook: Neutral / Uncertain"


# Main execution
if __name__ == "__main__":
    articles = fetch_news(query="stock market", max_articles=120)  # Get ~120
    if articles:
        news_list = []
        sentiments = []

        for article in articles:
            title = article.get("title", "")
            description = article.get("description", "")
            content = (title or "") + " " + (description or "")

            if not content.strip():
                continue

            sentiment = analyze_sentiment(content)
            sentiments.append(sentiment)

            news_list.append({
                "Title": title,
                "Description": description,
                "Sentiment": sentiment
            })

        if news_list:
            df = pd.DataFrame(news_list)
            print(df)

            # 🔽 Save results to CSV
            df.to_csv("news_sentiment_analysis.csv", index=False)
            print("\n✅ Data exported to news_sentiment_analysis.csv")
            print(f"✅ Total Articles Analyzed: {len(news_list)}")

            print("\n📈", predict_market(sentiments))
        else:
            print("No analyzable articles found.")
    else:
        print("No news data found.")


Enter your API key: ··········
Enter your URL key: ··········
                                                 Title  \
0    Arsenal Strengthen Defense as Mikel Arteta Sec...   
1    A $23 Trillion Cash Pile Holds Key for Chinese...   
2    Appeals court rules many of Trump's tariffs ar...   
3    Can China’s US$22 trillion savings and global ...   
4               Virgo Horoscope Today, August 30, 2025   
..                                                 ...   
115  Tesla asks court to throw out big damage award...   
116  Avista Makes Annual Price Adjustment Requests ...   
117  Most of Donald Trump's tariffs illegal, US cou...   
118  The stock market is showing it will punish any...   
119  Most Trump tariffs are not legal, appeals cour...   

                                           Description Sentiment  
0    Arsenal are set to bolster their defense after...  Positive  
1    China’s stock rally is set to get a boost from...  Negative  
2    A federal appeals court said many o